### test ff
Powered by [Eleonora Priori](https://www.ecoaz.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


The trick above, useful to avoid scrolling output windows, does not work with recent jupyter versions; use settings.

In [1]:
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)



#generate random seed
repast4py.random.init(rng_seed=12345)
rng = repast4py.random.default_rng 

In [2]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, sector: int, labor: int): #, capital:float, minOrderDuration:int,\
                 #maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 #assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 #sectorialClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid

        self.sector=sector
        self.labor=labor
        """
        self.capital=capital
        #self.capitalQ= 0
        self.unavailableLabor=0
        self.unavailableCapital=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        """

        
        


In [3]:

class Model:
    
    def __init__(self):
        
      
        runner.schedule_repeating_event(0.0,  1, self.interactingWithFirms)

        runner.schedule_stop(0)
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################

        fileName = input("file name? ")
        firmCount= int(input("how many firms? "))
        
        #importing csv file containing info about firms 
        with open(fileName, newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',')#, quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1 #to skip the row of the headers
            k=0
            #for each record in .csv
            
            # case ff_with_class_limits.csv OR ff_correction_upper_limit.csv
            # #,NACE definition,type,dimensional class,Share of firms,Share of firms per sbs sector,
            # sbs reference row,L min,L max,Firms in absolute numbers,Workers per class in absolute numbers,
            # Total Workers without >= 250 class,Firms only >= 250 class

            if fileName == "ff_with_class_limits.csv": #or fileName == "ff_correction_upper_limit.csv":
                for row in firmReader:
                    #print(row)
                    labor = 0 
                    if self.rowNumber>=0:
                        
                        sector = row[0]
                        
                        if row[4]=='': row[4]=0
                        for i in range(int(float(row[4]) * firmCount)// rankNum):
                            if row[0] == '1' or row[0] == '2' or row[0] == '3': #agri, silvi and fishing
                                randomizer = rng.uniform()
                                if randomizer <= 0.15: labor = 1
                                elif randomizer <= 0.95 : labor = 2
                                else: labor = 3
                            else:
                                if row[8] != '':
                                    if int(row[7]) == 0: row[7] = '1' #to avoid firms with 0 workers in the 0-9 class
                                    labor= rng.integers(int(row[7]), int(float(row[8]))+1) #+1 because integers exclude extremes                         
                                else: 
                                    labor = 0
                            """
                            capital= row[3] + rng.random()*(row[4] -row[3])
                            minOrderDuration= row[5]
                            maxOrderDuration= row[6]
                            recipe= row[7] #K/L 
                            laborProductivity= row[8]
                            maxOrderProduction= row[9]
                            avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                            plannedMarkup=row[11]
                            orderObservationFrequency=rng.integers(row[12], row[13]+1)
                            productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                            sectorialClass=int(self.rowNumber)"""
                            aFirm =Firm(k, rank, sector, labor) #, capital, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                    #maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                    #sectorialClass)
                            context.add(aFirm)
                            k += 1 # first element of the UID of the agents
                    self.rowNumber += 1
                self.firmCount=k #one more, here is a count, not an id
        
    #interactingWithFirms
    def interactingWithFirms(self):

        #print(self.firmCount)

        if self.firmCount==0:
            print("No firms created.")
        else:
            print(f"There are {len(list(context.agents(agent_type=0)))} firms in the simulation")
            
            count=0
            countFa=0
            countWa=0
            for aFirm in context.agents(agent_type=0):
                count+=aFirm.labor
                if int(aFirm.sector) <= 3: 
                    countFa+=1
                    countWa+=aFirm.labor
            print(f"with {count} workers.")

            print(f"Subset of the firms of Agriculture, Forestry and Fishing {countFa} "+\
                    f"with {countWa} workers")

            
            #Europe 23
            #Agriculture, Forestry and Fishing => AFF
            actual_firm_number=20751071
            actual_AFF_firm_number=10000000
            actual_worker_number=19000000+152533008 
            actual_AFF_ worker_number=19000000
            # from firm-features-generation.ipynb

            print(f"\nEurope (23 countries): actual firm number {actual_firm_number} "+\
                  f"with {actual_worker_number} workers.")
            print(f"Subset in Agriculture, Forestry and Fishing: {actual_AFF_ firm_number} firms "+\
                  f"with {actual_AFF_worker_number} workers.")

            expected_workers_in_simulation = actual_worker_number*len(list(context.agents(agent_type=0)))/\
                                             actual_firm_number
            expected_AFF_firm_number=
            print(f"Proportionally: expected number of workers in the simulation {int(expected_workers_in_simulation)};")
            print(f"                expected number of AFF firms in the simulation {int(expected_workers_in_simulation)}.")

    #finish
    def finish(self):
        print("\nConcluded")
    
    def start(self):
        runner.execute()

In [4]:
def run():

    model = Model() 
    model.start()
    
run()

file name?  ff_with_class_limits.csv
how many firms?  1000000


There are 999622 Firms in the simulation
with 8486067 workers.
Subset of the firms of Agriculture, Forestry and Fishing 481898 with 915489 workers

Europe (23 countries): actual firm number 20751071 with 171533008 workers.
Subset in Agriculture, Forestry and Fishing: 10000000 firms with 19000000 workers.
Proportionally, expected number of workers in the simulation 8263099.

Concluded


In [5]:
#context.agents(agent_type=0)